In [ ]:
import pandas as pd
import os

In [ ]:
csv_path = os.path.join(os.getcwd(), 'src', 'DataSet', 'heart.csv')

df = pd.read_csv(csv_path)

df_copy = df.copy()


In [ ]:
print(df_copy.shape)
print(df_copy.dtypes)
print(df_copy.isnull().sum())
print(df_copy.describe())
print(df_copy['HeartDisease'].value_counts())